#  Lasso Regression

* The aim is to find the coefficients of independent variables that minimize the Root Mean Square Error(RMSE) by applying a penalty to these coefficients.



* Lasso Regression is recommended to overcome the disadvantage of ridge regression keeping all related and unrelated independent variables in the model.



* If the alpha value is chosen big enough, it makes some coefficients zero. Thus, it will make the variable selection


* Optimum selection of the alpha is very important.For this reason , we choose the most optimum alpha by using the cross validation method.



* Select the alpha that gives the smallest Root Mean Square Error value. 



* Finally, with this selected alpha, the model is fitted to all observations again. 

## 1-)Data Preprocessing

In [1]:
import pandas as pd
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
X_train,X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

print("X_train", X_train.shape)

print("y_train",y_train.shape)

print("X_test",X_test.shape)

print("y_test",y_test.shape)

training = df.copy()

print("training", training.shape)

X_train (197, 19)
y_train (197,)
X_test (66, 19)
y_test (66,)
training (263, 20)


## 2-)Model

In [4]:
from sklearn.linear_model import Lasso

In [5]:
lasso_model = Lasso(alpha = 0.1).fit(X_train, y_train)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8249174.751388096, tolerance: 3898.686956380658
  positive)


In [6]:
lasso_model

Lasso(alpha=0.1)

* We chose ourselves alpha = 0.1 
    

* We don't know whether this value(alpha = 0.1 ) is the best or not.


* We will try to find the optimum value of alpha with the cross validation method.That's why we will tune the model 

In [7]:
lasso_model.coef_

array([-1.72206506e+00,  8.56210197e+00,  6.91175137e+00, -3.13240128e+00,
       -2.00771676e+00,  5.36159035e+00,  8.86871593e+00, -1.69520371e-01,
       -5.58121413e-02, -3.53962588e-01,  1.70961000e+00,  6.40603469e-01,
       -6.58519895e-01,  2.60093222e-01,  2.78717030e-01, -5.92690965e-01,
        1.12659630e+02, -9.99652090e+01, -4.81289395e+01])

In [8]:
lasso_model.intercept_

-7.7694903740984955

## 3-) Prediction

In [9]:
y_pred = lasso_model.predict(X_test)
y_pred[0:5]


array([ 613.88833029,  701.97056731, 1005.55539526,  414.08312603,
        399.18417127])

In [10]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [11]:
test_error_before=np.sqrt(mean_squared_error(y_test, y_pred))
test_error_before# test error before model tuning

356.7545270148768

## 4-)Model tuning

* We will try to find the optimum alpha value using the cros validation method.




* In other words, instead of choosing the alpha value randomly ,which is called a hyperparameter we will find it with the cross validation method.



* Finally, we will create the final model by using the most optimum alpha value.

### 4.1-) Croos Validation Method

In [12]:
from sklearn.linear_model import LassoCV

In [13]:
lasso_cv_model = LassoCV(alphas = None, 
                         cv = 10, 
                         max_iter = 10000, 
                         normalize = True)

In [14]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=10, max_iter=10000, normalize=True)

In [15]:
lasso_cv_model.alpha_ #the most optimum alpha value 

0.39406126432470073

### 4-2) Fitting final model by using optimum alpha value

In [16]:
lasso_tuned = Lasso(alpha = lasso_cv_model.alpha_)

In [17]:
lasso_tuned.fit(X_train, y_train)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8252823.943523985, tolerance: 3898.686956380658
  positive)


Lasso(alpha=0.39406126432470073)

In [18]:
y_pred1 = lasso_tuned.predict(X_test)
y_pred1[0:5]

array([ 612.35137732,  700.33250549, 1006.70519014,  413.47629461,
        402.48498299])

In [19]:
test_error_after=np.sqrt(mean_squared_error(y_test, y_pred1))
test_error_after #test error after tuning

356.5226376958367